In [5]:
%pip install SpeechRecognition 
%pip install pydub
%pip install openai-whisper
%pip install ffmpeg

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [11]:
%pip install "numpy<2"

  Using cached numpy-1.26.4-cp39-cp39-macosx_10_9_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp39-cp39-macosx_10_9_x86_64.whl (20.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.18 requires flatbuffers>=2.0, but you have flatbuffers 1.12 which is incompatible.
mediapipe 0.10.18 requires protobuf<5,>=4.25.3, but you have protobuf 3.20.3 which is incompatible.
tensorflow 2.5.0 requires numpy~=1.19.2, but you have numpy 1.26.4 which is incompatible.
tensorflow 2.5.0 requires typing-extensions~=3.7.4, but you have typing-extensions 4.12.2 which is incompatible.
tf-keras 2.16.0 requires tensorflow<2.17,>=2.16, but you have tensorflow 2.5.0 which is incompatible.
Note: you may need to resta

In [ ]:
#conda install -c conda-forge ffmpeg
#need to run this for ffmpeg to work

In [ ]:
import speech_recognition as sr


from pydub import AudioSegment
from pydub.utils import which

AudioSegment.converter = which("ffmpeg")
AudioSegment.ffprobe = which("ffprobe")


# Initialize the recognizer
recognizer = sr.Recognizer()

# Specify the path to your MP3 file
mp3_file = "/Users/rgopalam/Desktop/office/audio2.mp3"

# Convert MP3 to WAV
from pydub import AudioSegment
audio = AudioSegment.from_mp3(mp3_file)
wav_file = "temp_audio.wav"
audio.export(wav_file, format="wav")

# Recognize the speech in the audio file
import whisper

def transcribe_audio(audio_file):
    model = whisper.load_model("base")
    result = model.transcribe(audio_file)
    return result["text"]

transcript = transcribe_audio(wav_file)
print(transcript)



RuntimeError: Numpy is not available

In [16]:
import os
from pydub import AudioSegment
from pydub.utils import which
from google.cloud import speech

# Ensure ffmpeg and ffprobe paths are set
AudioSegment.converter = which("ffmpeg")
AudioSegment.ffprobe = which("ffprobe")

# Set the path to your Google Cloud JSON credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/rgopalam/Desktop/seismic-rarity-427422-p7-ab3b4a8726ef.json"

def get_sample_rate(audio_file):
    """Get the sample rate of the audio file."""
    try:
        audio = AudioSegment.from_file(audio_file)
        return audio.frame_rate
    except Exception as e:
        print(f"Error getting sample rate: {e}")
        return None

def convert_mp3_to_wav(mp3_file, output_file="temp_audio.wav"):
    """Convert an MP3 file to WAV format with a single channel (mono)."""
    try:
        audio = AudioSegment.from_mp3(mp3_file)
        audio = audio.set_channels(1)  # Convert to mono
        audio.export(output_file, format="wav")
        return output_file
    except Exception as e:
        print(f"Error converting MP3 to WAV: {e}")
        return None

def transcribe_audio_google(audio_file):
    """Transcribe audio using Google Cloud Speech-to-Text."""
    try:
        # Initialize the Speech-to-Text client
        client = speech.SpeechClient()

        # Read the audio file
        with open(audio_file, "rb") as audio:
            audio_content = audio.read()

        # Get the sample rate of the WAV file
        sample_rate = get_sample_rate(audio_file)
        if not sample_rate:
            print("Could not determine sample rate.")
            return None

        # Configure the recognition request
        audio_config = speech.RecognitionConfig(
            encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
            sample_rate_hertz=sample_rate,  # Dynamically match the WAV file's sample rate
            language_code="en-US"
        )
        audio_request = speech.RecognitionAudio(content=audio_content)

        # Perform the transcription
        response = client.recognize(config=audio_config, audio=audio_request)

        # Concatenate all transcriptions into one string
        transcript = " ".join([result.alternatives[0].transcript for result in response.results])
        return transcript

    except Exception as e:
        print(f"Error during transcription: {e}")
        return None

# Main execution
if __name__ == "__main__":
    mp3_file = "/Users/rgopalam/Desktop/office/audio2.mp3"
    
    if not os.path.exists(mp3_file):
        print(f"File not found: {mp3_file}")
    else:
        wav_file = convert_mp3_to_wav(mp3_file)
        if wav_file:
            transcript = transcribe_audio_google(wav_file)
            if transcript:
                print("Transcription:")
                print(transcript)
            # Clean up temporary file
            os.remove(wav_file)


Transcription:
from this now I'm loving my shoes these aren't my shoes I'm buying them so you can do whatever you want to them mate go on get in there  get in there you love the fact that this dog is chewing on these  they're tasty they're shoes how much are these shoes 1,800 bucks I'm getting there guys
